In [ ]:
# Install dependencies if needed
# The torch-nos package provides tools for reading videos
!pip install requests Pillow torch-nos pytube
!conda install loguru -y

In [ ]:
import os
import requests


VLM_BASE_URL = "https://vlm-dev.nos.run/v1"
response = requests.get(f"{VLM_BASE_URL}/health")
response.raise_for_status()
assert response.status_code == 200

In [ ]:
VLM_API_KEY = os.getenv("VLM_API_KEY", None)
if VLM_API_KEY is None:
    VLM_API_KEY = input()
print(f"Using API key: {VLM_API_KEY[:4]}...")

In [ ]:
def yt_playlist(playlist_url: str, start: int = 0, end: int = 100) -> list:
    """Get all URLs from a YouTube playlist"""
    playlist = Playlist(playlist_url)
    # This forces pytube to fetch all the video URLs in the playlist
    playlist._video_regex = None
    return list(playlist.video_urls)[start:end]

In [ ]:
from IPython.display import YouTubeVideo

# YC Convo
url = "https://www.youtube.com/watch?v=KxjPgGLVJSg"
YouTubeVideo(url)

In [ ]:
# Test the video url -> audio API
from fastapi import status
import json

headers = {
    "Content-Type": "application/json",
    "X-API-Key": VLM_API_KEY,
}

# Generate audio transcription
json_data = {
    "url": url,
    "model": "vlm-1",
    "domain": "video.transcription",
}
response = requests.post(f"{VLM_BASE_URL}/video/generate", headers=headers, json=json_data)
assert response.status_code == status.HTTP_200_OK, f"Response failed: {response.text}"
json_response = response.json()
print(f"Transcription: {json.dumps(json_response, indent=2)}")